In [22]:
import pandas as pd
import numpy as np
import time
import iddModel.learnUsingNumpy as idoctor
import iddModel.doctor as idoc
import multiprocessing
import parmap 
import math
pd.set_option('display.max_row', 1200)
pd.set_option('display.max_columns', 100)

In [23]:
tuningSetAgeRate = [[25,40,26.5,1.1],
                      [27,40,27,11.1],
                      [26,40,27,11.1]]
tuningSetRetireRate = [1.09,35,1]
%time npCostData = idoctor.getCost([tuningSetAgeRate, tuningSetRetireRate])
pd.DataFrame(npCostData)

CPU times: user 25.1 ms, sys: 2.82 ms, total: 27.9 ms
Wall time: 22.6 ms


,0,1,2,3,4,5,6,7,8,9,10,11
0,99,2958,4804,653,2667,4343,0.0281247,0.0341738,0.0492188,0.0177174,0.0387197,0.0550803
1,3132,1627,331,650,306,218,0.0281247,0.0188107,0.000133866,0.00502674,0.00238592,0.00203524
2,360,2093,3608,672,1745,3644,0.0294669,0.0232762,0.0349341,0.0162038,0.0262991,0.0451743
3,3237,1195,168,998,569,307,0.0294669,0.0133539,0.00165196,0.0092133,0.00547562,0.00307611
4,1895,885,1872,794,970,2888,0.0281119,0.00962396,0.019071,0.011137,0.0125249,0.0331449
5,2713,593,469,1310,855,424,0.0281119,0.00648723,0.00627709,0.0140236,0.00926054,0.00461761
6,2423,227,935,815,217,2099,0.030534,0.00266782,0.00912944,0.0102078,0.00335098,0.0234369
7,2916,24,482,1717,1156,549,0.030534,0.000369056,0.00588322,0.0184008,0.0124566,0.0059289
8,3003,1400,6,796,394,1195,0.0320998,0.0149748,5.77276e-05,0.00850742,0.00422048,0.0127803
9,3000,865,326,2060,1446,685,0.0320998,0.00925232,0.00348366,0.0220385,0.0154693,0.00732802


In [24]:
def runGetCost(tuningValueList):
    resultData = []
    
    for value in tuningValueList:
        resultData.append([value ,idoctor.getCost(value)])
        
        
    return resultData

In [55]:
def startGetCost(inputData):
    tRate1 = inputData[0]
    tRate2 = inputData[1]
    tRate3 = inputData[2]
    tRetire1 = inputData[3]
    tRetire2 = inputData[4]
    
    dataSize = len(tRate1)*len(tRate2)*len(tRate3)*len(tRetire1)*len(tRetire2)
    timeMinute = dataSize*22/1000/60/22
    curentData = 0
    resultData = []
    
    num_cores = 30

    now = time.localtime()
    print("│ Start Time : %04d/%02d/%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour+9, now.tm_min, now.tm_sec))
    print("│ expect running time : " + str(round(timeMinute)) + "min")
    print("│ Data Size = " + str(dataSize))
    print("└───────────────────────")
    print('│ '+str(round(curentData/dataSize*100,2))+'%')
    
    tuningList = []
    
    for b1 in tRate1:
        for b2 in tRate2:
            for b3 in tRate3:
                for c1 in tRetire1:
                    for c2 in tRetire2:
                        ageTunList = [[25,40,26,b1],
                                      [27,40,28,b2],
                                      [26,40,27,b3]]
                        retireTunList = [c1,35,c2]
                        tuningList.append([ageTunList,retireTunList])

    
    splitedTuningList = np.array_split(tuningList, num_cores) 
    splitedTuningList = [i.tolist() for i in splitedTuningList] 
    
    result = parmap.map(runGetCost, splitedTuningList, pm_pbar=True, pm_processes=num_cores)
    
    print("| End Time : %04d/%02d/%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour+9, now.tm_min, now.tm_sec))
    return result

In [56]:
tRate1 = np.arange(20, 110, 10)   # 연령분포모델 변수2 c/ 범위 0.3~1
tRate2 = np.arange(100, 1000, 100)   # 연령분포모델 변수2 c/ 범위 0.3~1
tRate3 = np.arange(100, 1000, 100)   # 연령분포모델 변수2 c/ 범위 0.3~1
tRetire1 = np.arange(1.1, 1.55, 0.05)  # 은퇴율모델 변수1 / 범위 1 이상
tRetire2 = np.arange(0.5, 10.5, 0.5) # 은퇴율모델 변수3 / 범위 0 이상
    
liCostData = startGetCost([tRate1, tRate2, tRate3, tRetire1, tRetire2])

│ Start Time : 2020/11/17 21:24:46
│ expect running time : 2min
│ Data Size = 131220
└───────────────────────
│ 0.0%


100%|██████████| 30/30 [03:10<00:00,  6.35s/it]


| End Time : 2020/11/17 21:24:46


In [66]:
npCostData = np.array(liCostData)

In [67]:
np.shape(npCostData)

(30, 4374, 2)

In [68]:
minManCost = (np.concatenate((npCostData[0][0][1][0],npCostData[0][0][1][2],npCostData[0][0][1][4],npCostData[0][0][1][6],npCostData[0][0][1][8],npCostData[0][0][1][10]))).reshape(6,12).sum(axis=0)
minWomanCost = (np.concatenate((npCostData[0][0][1][1],npCostData[0][0][1][3],npCostData[0][0][1][5],npCostData[0][0][1][7],npCostData[0][0][1][9],npCostData[0][0][1][11]))).reshape(6,12).sum(axis=0)
minManTuningValue = npCostData[0][0][0]
minWomanTuningValue = npCostData[0][0][0]
count = 0
for data1 in npCostData:
    for data2 in data1:   
        count += 1
        npManCost = (np.concatenate((data2[1][0],data2[1][2],data2[1][4],data2[1][6],data2[1][8],data2[1][10]))).reshape(6,12).sum(axis=0)
        npWomanCost = (np.concatenate((data2[1][1],data2[1][3],data2[1][5],data2[1][7],data2[1][9],data2[1][11]))).reshape(6,12).sum(axis=0)

        manCostCount = 0
        womanCostCount = 0
        
        for i in range(6,12):
            if npManCost[i]<=minManCost[i]:
                manCostCount += 1
            else:
                break
                
        for i in range(6,12):
            if npWomanCost[i]<=minWomanCost[i]:
                womanCostCount += 1
            else:
                break
                
        if manCostCount==6:
            minManCost = npManCost
            minManTuningValue = data2[0]
            
        if womanCostCount==6:
            minWomanCost = npWomanCost
            minWomanTuningValue = data2[0]

pd.DataFrame([[minManCost, minManTuningValue],[minWomanCost,minWomanTuningValue]])
print(count)

131220


In [69]:
print(minManCost)
print()
print(minManTuningValue)
print()
print(minWomanCost)
print()
print(minWomanTuningValue)
pd.DataFrame([[minManCost, minManTuningValue],[minWomanCost,minWomanTuningValue]])

[24643.0 12766.0 21541.0 2641.0 8531.0 8396.0 0.15455178799189362
 0.13346260061528456 0.13556896142777886 0.16021811202889943
 0.1300745402472954 0.11388573425688236]

[[[25, 40, 26, 20], [27, 40, 28, 100], [26, 40, 27, 100]], [1.1, 35, 0.5]]

[27240.0 7648.0 3721.0 6373.0 5563.0 7181.0 0.15455178799189356
 0.08040008991283004 0.02147444995157037 0.03906107472226163
 0.044060245568117565 0.06654088827206679]

[[[25, 40, 26, 20], [27, 40, 28, 100], [26, 40, 27, 100]], [1.1, 35, 0.5]]


,0,1
0,"[24643.0, 12766.0, 21541.0, 2641.0, 8531.0, 83...","[[[25, 40, 26, 20], [27, 40, 28, 100], [26, 40..."
1,"[27240.0, 7648.0, 3721.0, 6373.0, 5563.0, 7181...","[[[25, 40, 26, 20], [27, 40, 28, 100], [26, 40..."


In [5]:
tuningSetAgeRate = [[25,40,26.5,1.1],
                      [27,40,27,11.1],
                      [26,40,27,11.1]]
tuningSetRetireRate = [1.09,35,1]


#의대수, 의전원수, 의대남비율, 의전원남비율 
npBasicPopulation = np.zeros([22,4])
for i in range(22):
    npBasicPopulation[i] = np.array([3400,50,0.6,0.6])
    
data = idoc.makeDoctorData(npBasicPopulation, [tuningSetAgeRate,tuningSetRetireRate])

/root/project/doctorPrj/IITPMainProject/jupyterNotebook/kim/iddModel/doctor.py:61: RuntimeWarning: invalid value encountered in true_divide
  npRateData = npData / sumValue


TypeError: 'float' object cannot be interpreted as an integer

In [45]:
idoctor.makeLogModel2([27,40,29,10])[25:30].sum()

0.47249500786521914

In [47]:
dataList = []
for m in range(27,30):
    for n in range(1,2000,10):
            dataList.append([idoctor.makeLogModel2([26, 40, m, n]), [m,n]])
dataList

[[array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan]),
  [27, 1]],
 [array([0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.16353432, 0.18039783, 0.14697273, 0.11917779,


In [48]:
sumList = []
for data in dataList:
    sumList.append([(data[0][25:30].sum()),data[1][0], data[1][1]])
sumList2 = np.array(sumList).T

In [49]:
sumList2[0]

array([       nan, 0.61008266, 0.65163921, 0.67621093, 0.69344984,
       0.70661137, 0.71718774, 0.72598531, 0.73348802, 0.74000846,
       0.74575982, 0.75089379, 0.75552197, 0.75972875, 0.76357944,
       0.76712556, 0.77040844, 0.7734617 , 0.77631307, 0.77898564,
       0.78149886, 0.78386923, 0.78611088, 0.78823601, 0.79025518,
       0.79217764, 0.79401149, 0.79576389, 0.7974412 , 0.79904905,
       0.80059251, 0.80207612, 0.80350396, 0.80487972, 0.80620676,
       0.80748811, 0.80872657, 0.80992466, 0.81108471, 0.81220885,
       0.81329906, 0.81435715, 0.81538479, 0.81638353, 0.81735482,
       0.81829999, 0.8192203 , 0.82011689, 0.82099086, 0.82184322,
       0.82267492, 0.82348685, 0.82427984, 0.82505468, 0.82581211,
       0.82655282, 0.82727747, 0.82798669, 0.82868104, 0.8293611 ,
       0.83002739, 0.83068039, 0.83132059, 0.83194842, 0.83256433,
       0.8331687 , 0.83376193, 0.83434437, 0.83491639, 0.83547832,
       0.83603046, 0.83657313, 0.83710662, 0.8376312 , 0.83814

In [50]:
import plotly.graph_objects as go
import numpy as np

# Helix equation
t = np.linspace(0, 20, 100)
x, y, z = np.cos(t), np.sin(t), t

fig = go.Figure(data=[go.Scatter3d(
    x=sumList2[1],
    y=sumList2[2],
    z=sumList2[0],
    mode='markers',
    marker=dict(
        size=3,
        color=z,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)])

# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()
